In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
wine_quality_df = pd.read_csv("/kaggle/input/wine-quality-dataset/WineQT.csv", index_col = "Id")
wine_quality_df.head()

In [ ]:
wine_quality_df.info()

In [ ]:
# check distribution of quality

wine_quality_df["quality"].hist()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(wine_quality_df.drop("quality", axis = 1), 
                                                    wine_quality_df["quality"],
                                                    test_size = 0.2, stratify = wine_quality_df["quality"])

In [ ]:
# Let's plot relationship between wine quality and each input feature
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize = (20, 20))

for n, column in enumerate(X_train.columns):
    plt.subplot(4, 3, n + 1)
    sns.stripplot(x = y_train, y = X_train[column], size = 1.5)

In [ ]:
#polynomial regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

polynomial_regressor = Pipeline([
    ("poly_features", PolynomialFeatures(2)),
    ("lin_reg", LinearRegression())
])

polynomial_regressor.fit(X_train, y_train)

In [ ]:
simple_predictions = polynomial_regressor.predict(X_test)
simple_predictions[:5], y_test[:5]

In [ ]:
#Ridge regression model uses l2 regularization to reduce overfitting.

from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

ridge_regressor = Pipeline([
    ("poly_features", PolynomialFeatures(2)),
    ("scaler", StandardScaler()),
    ("ridge_reg", Ridge(alpha = 1))
])

ridge_regressor.fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV

ridge_regressor = Pipeline([
    ("poly_features", PolynomialFeatures()),
    ("scaler", StandardScaler()),
    ("ridge_reg", Ridge())
])

param_grid = {'poly_features__degree' : [1, 2, 3, 4],
              'ridge_reg__alpha' : [1.0, 10.0, 100.0, 1000.0]}

search = GridSearchCV(estimator = ridge_regressor,
                     param_grid = param_grid, 
                     verbose = 1, cv = 5,
                     scoring = "neg_mean_absolute_error")

search.fit(X_train, y_train)

In [ ]:
search.best_params_, search.best_score_